<a href="https://colab.research.google.com/github/caesar-one/audio-transformer/blob/master/raw_transformer/Audio_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Download Dataset
!wget https://storage.googleapis.com/urbansound8k/UrbanSound8K.tar.gz
!tar -xf UrbanSound8K.tar.gz
!rm UrbanSound8K.tar.gz

--2020-05-15 10:44:46--  https://storage.googleapis.com/urbansound8k/UrbanSound8K.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/gzip]
Saving to: ‘UrbanSound8K.tar.gz’

UrbanSound8K.tar.gz 100%[===================>]   5.61G  42.8MB/s    in 2m 2s   

2020-05-15 10:46:48 (47.2 MB/s) - ‘UrbanSound8K.tar.gz’ saved [6023741708/6023741708]



In [0]:
# Clone repo
!git clone https://github.com/caesar-one/audio-transformer
!mv -v audio-transformer/* .
!rm -rf audio-transformer

Cloning into 'audio-transformer'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.
renamed 'audio-transformer/dataset.py' -> './dataset.py'
renamed 'audio-transformer/LICENSE' -> './LICENSE'
renamed 'audio-transformer/main.py' -> './main.py'
renamed 'audio-transformer/models.py' -> './models.py'
renamed 'audio-transformer/README.md' -> './README.md'
renamed 'audio-transformer/requirements.txt' -> './requirements.txt'


In [0]:
IN_COLAB = True
try:
    from google.colab import drive
except:
    IN_COLAB = False
if IN_COLAB: 
    drive.mount('/content/drive')
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
import time
import math
import h5py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from dataset import AudioDataset#, get_proper_shape
import dataset
from models import AudioTransformer
from sklearn.metrics import classification_report
from glob import glob

if torch.cuda.is_available():
    print("Cuda device:", torch.cuda.get_device_name())

# Trains a single epoch with hyper-parameters provided
def train(model, criterion, optimizer, data, device):
    model.train() # Turn on the train mode
    total_loss = 0.
    progress = tqdm(data, "Training")
    for X, y in progress:
        X = X.permute((1,0,2)).to(device) # convert to seq first shape and move to desired device
        y = y.long()
        optimizer.zero_grad()
        output = model(X)
        output = output.cpu()
        output_flat = output.view(-1, num_classes)
        loss = criterion(output_flat, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += len(X) * loss.item()
    return total_loss / len(data.dataset)

def evaluate(model, criterion, data, device):
    model.eval() # Turn on the evaluation mode
    total_loss = 0.
    y_true, y_pred = [], []
    with torch.no_grad():
        for X, y in tqdm(data, "Evaluation"):
            X = X.permute((1,0,2)).to(device) # convert to seq first shape and move to desired device
            y = y.long()
            output = model(X)
            output = output.cpu()
            output_flat = output.view(-1, num_classes)
            y_true += y.tolist()
            y_pred += torch.argmax(output_flat, -1).tolist()
            total_loss += len(X) * criterion(output_flat, y).item()
    result = classification_report(y_true, y_pred, target_names=classes, digits=4, output_dict=True)
    result["loss"] = total_loss / len(data.dataset)
    result["pretty"] = classification_report(y_true, y_pred, target_names=classes, digits=4, output_dict=False)
    return result
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 20
eval_batch_size = 10
dataset_path = ""
dataset_cache_name = "UrbanSound8k_cache.h5"
drive_path = "/content/drive/My Drive/"

classes = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling',
                'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']

num_classes = len(classes)

model = AudioTransformer(d_model=256, nhead=8, dim_feedforward=1024, num_layers=6, num_classes=num_classes, dropout=0.1)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#Loop over epochs. Save the model if the validation loss is the best we’ve seen so far. Adjust the learning rate after each epoch.
best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None


if dataset_cache_name not in glob("*.h5"):
    print("No dataset cache in runtime, ",end="")
    if drive_path + dataset_cache_name not in glob(drive_path + "*.h5"):
        print("creating... ", end="")
        dataset_filename = dataset.load(dataset_path, dataset_cache_name, False)
        !cp {dataset_cache_name} /content/drive/My\ Drive/
    else:
        print("copying from Drive... ", end="")
        !cp /content/drive/My\ Drive/{dataset_cache_name} .
    print("Done!")

audio_data = h5py.File(dataset_cache_name, "r")
group = audio_data["urban_sound_8k"]

X_train = group["X_train"]
X_val = group["X_val"]
X_test = group["X_test"]
y_train = group["y_train"]
y_val = group["y_val"]
y_test = group["y_test"]

load_in_RAM = True
if load_in_RAM:
    X_train = X_train[:]
    X_val = X_val[:]
    X_test = X_test[:]
    y_train = y_train[:]
    y_val = y_val[:]
    y_test = y_test[:]
    audio_data.close()

train_dataloader = DataLoader(AudioDataset(X_train, y_train), batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(AudioDataset(X_val, y_val), batch_size, shuffle=False, drop_last=True)
test_dataloader = DataLoader(AudioDataset(X_test, y_test), batch_size, shuffle=False, drop_last=True)
model.to(device)
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train_loss = train(model, criterion, optimizer, train_dataloader, device)
    print("Train Loss:", train_loss)
    val_results = evaluate(model, criterion, val_dataloader, device)
    print("Val Loss:",val_results["loss"],"\nClassification report:\n",val_results["pretty"])
    scheduler.step()

    #print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
    #      'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
    #                                 val_loss, math.exp(val_loss)))

if not load_in_RAM:
    audio_data.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cuda device: Tesla K80



Train Loss: 13.80932935086798



Val Loss: 14.49054155922434 
Classification report:
                   precision    recall  f1-score   support

 air_conditioner     0.1600    0.1809    0.1698       199
        car_horn     0.8857    0.3039    0.4526       102
children_playing     0.5217    0.1818    0.2697       198
        dog_bark     0.6442    0.5303    0.5817       198
        drilling     0.4195    0.5628    0.4807       199
   engine_idling     0.4946    0.4292    0.4596       212
        gun_shot     0.3681    0.9054    0.5234        74
      jackhammer     0.2050    0.2059    0.2055       238
           siren     0.5819    0.8688    0.6970       282
    street_music     0.4000    0.2323    0.2939       198

        accuracy                         0.4305      1900
       macro avg     0.4681    0.4401    0.4134      1900
    weighted avg     0.4530    0.4305    0.4126      1900




Train Loss: 10.116552480390366



Val Loss: 15.260306376389984 
Classification report:
                   precision    recall  f1-score   support

 air_conditioner     0.0820    0.0251    0.0385       199
        car_horn     0.6630    0.5980    0.6289       102
children_playing     0.5729    0.2778    0.3741       198
        dog_bark     0.6571    0.5808    0.6166       198
        drilling     0.3931    0.6281    0.4836       199
   engine_idling     0.2256    0.5991    0.3277       212
        gun_shot     0.4565    0.8514    0.5943        74
      jackhammer     0.1525    0.1134    0.1301       238
           siren     0.7851    0.6738    0.7252       282
    street_music     0.4474    0.0859    0.1441       198

        accuracy                         0.4132      1900
       macro avg     0.4435    0.4433    0.4063      1900
    weighted avg     0.4387    0.4132    0.3903      1900



...


Val Loss: 23.209686850163084 
Classification report:
                   precision    recall  f1-score   support

 air_conditioner     0.6050    0.3618    0.4528       199
        car_horn     0.5396    0.7353    0.6224       102
children_playing     0.4700    0.6717    0.5530       198
        dog_bark     0.6739    0.7828    0.7243       198
        drilling     0.4378    0.5477    0.4866       199
   engine_idling     0.5460    0.4198    0.4747       212
        gun_shot     0.8375    0.9054    0.8701        74
      jackhammer     0.3967    0.2017    0.2674       238
           siren     0.8711    0.8865    0.8787       282
    street_music     0.5284    0.6111    0.5667       198

        accuracy                         0.5889      1900
       macro avg     0.5906    0.6124    0.5897      1900
    weighted avg     0.5850    0.5889    0.5747      1900




Train Loss: 0.017946259953003137



Val Loss: 23.24894257180062 
Classification report:
                   precision    recall  f1-score   support

 air_conditioner     0.6083    0.3668    0.4577       199
        car_horn     0.5357    0.7353    0.6198       102
children_playing     0.4769    0.6768    0.5595       198
        dog_bark     0.6624    0.7828    0.7176       198
        drilling     0.4337    0.5427    0.4821       199
   engine_idling     0.5460    0.4198    0.4747       212
        gun_shot     0.8375    0.9054    0.8701        74
      jackhammer     0.3866    0.1933    0.2577       238
           siren     0.8720    0.8936    0.8827       282
    street_music     0.5378    0.6111    0.5721       198

        accuracy                         0.5895      1900
       macro avg     0.5897    0.6128    0.5894      1900
    weighted avg     0.5840    0.5895    0.5746      1900




Train Loss: 0.023488100282497437



Val Loss: 23.31031221951268 
Classification report:
                   precision    recall  f1-score   support

 air_conditioner     0.5983    0.3518    0.4430       199
        car_horn     0.5396    0.7353    0.6224       102
children_playing     0.4735    0.6768    0.5572       198
        dog_bark     0.6710    0.7828    0.7226       198
        drilling     0.4360    0.5477    0.4855       199
   engine_idling     0.5394    0.4198    0.4721       212
        gun_shot     0.8375    0.9054    0.8701        74
      jackhammer     0.3917    0.1975    0.2626       238
           siren     0.8720    0.8936    0.8827       282
    street_music     0.5354    0.6111    0.5708       198

        accuracy                         0.5889      1900
       macro avg     0.5894    0.6122    0.5889      1900
    weighted avg     0.5836    0.5889    0.5740      1900



In [0]:
print(val_results["loss"])

16.858138675540296
